# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity_nosql
WITH REPLICATION = 
{'class' : 'SimpleStrategy', 'replication_factor' : 1}""")

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity_nosql')

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Question 1

#### When working with a NoSQL database, we should think about the query first. 

#### (1) Expected output columns - artist, song and length of the track
#### (2) The output should be based on sessionId and itemInSession.

#### According to the expectation, the query should be:

``` 
    SELECT artist, song, length 
    FROM TABLE_NAME 
    WHERE sessionId = value AND itemInSession = value 
```

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "SELECT artist, song, length \
          FROM music_app_history \
          WHERE sessionId = 338 AND itemInSession = 4"


#### Next, we create the table music_app_history, and insert data into the table. 

In [9]:
# Create table1
session.execute("DROP TABLE IF EXISTS music_app_history ")
table1_creation = """CREATE TABLE IF NOT EXISTS music_app_history """
table1_creation += "(sessionId int, itemInSession int, \
                    artist text, song text, length float,  \
                    PRIMARY KEY (sessionID, itemInSession))"

session.execute(table1_creation)

In [10]:
# Insert table1 
file = 'event_datafile_new.csv'

query_insert1 = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length)"
query_insert1 += " VALUES(%s, %s, %s, %s, %s)"
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        session.execute(query_insert1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Finally, we execute the query to validate the results.

In [11]:
rows = session.execute(query1)

# print("{:<14s} {:<40s} {:<14s}".format('artist', 'Song', 'length'))
    
# for row in rows:
#     print("{:<14s} {:<40s} {:<14f}".format(row.artist, row.song, row.length))

tbl = PrettyTable(['artist', 'Song', 'length'])
for row in rows:
    tbl.add_row([row.artist, row.song, row.length])

print(tbl)

+-----------+---------------------------------+--------------------+
|   artist  |               Song              |       length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


## Question 2

#### Think about the query first. 

#### (1) Expected output columns - artist, song (sorted by itemInSession) and user's first name and last name
#### (2) The output should satisfy userid = 10 and sessionid = 182. 
#### (3) The above entails that the primary key should contain a composite partition key (userid, sessionid), and a clustering column (itemInSession).

#### According to the expectation, the query should be:

``` 
    SELECT artist, song, firstName, lastName, itemInSession
    FROM music_app_history2 
    WHERE userId = 10 AND sessionId = 182
```

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "SELECT artist, song, firstName, lastName, itemInSession\
          FROM music_user_session \
          WHERE userId = 10 AND sessionId = 182 "

#### Next, we create the table music_app_history, and insert data into the table. 
#### Then, we execute the query to validate the results.

In [13]:
# Create table2 
session.execute("DROP TABLE IF EXISTS music_user_session ")
table2_creation = """CREATE TABLE IF NOT EXISTS music_user_session """
table2_creation += "(userId int, sessionId int, itemInSession int, \
                     artist text, song text, firstName text, lastName text, \
                     PRIMARY KEY ((userId, sessionId), itemInSession)) "

session.execute(table2_creation)

In [14]:
# Insert table2
query_insert2 = "INSERT INTO music_user_session (userId, sessionId, itemInsession, artist, song, firstName, lastName) "
query_insert2 += " VALUES(%s, %s, %s, %s, %s, %s, %s)"

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute(query_insert2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [15]:
rows2 = session.execute(query2)

# print("{:<14s} {:<18s} {:<55s} {:<14s} {:<14s}".format('iteminsession', 'artist', 'Song', 'First Name', 'Last Name'))
# for row in rows2:
#     print("{:<14d} {:<18s} {:<55s} {:<14s} {:<14s}".format(row.iteminsession, row.artist, row.song, row.firstname, row.lastname))
    
tbl = PrettyTable(['iteminsession', 'artist', 'Song', 'First Name', 'Last Name'])
for row in rows2:
    tbl.add_row([row.iteminsession, row.artist, row.song, row.firstname, row.lastname])
print(tbl)

+---------------+-------------------+------------------------------------------------------+------------+-----------+
| iteminsession |       artist      |                         Song                         | First Name | Last Name |
+---------------+-------------------+------------------------------------------------------+------------+-----------+
|       0       |  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|       1       |    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
|       2       | Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|       3       |   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+---------------+-------------------+------------------------------------------------------+------------+-----------+


## Question 3

#### Think about the query first. 

#### (1) Expected output columns - user's first name and last name, song
#### (2) The output should satisfy song = 'All Hands Against His Own'. 
#### (3) The above entails that the primary key should be (song, userid) or (song, userid, firstName, lastName). We choose the former case as it is sufficient. 

#### According to the expectation, the query should be:

```
    SELECT firstName, lastName, song 
    FROM music_app_history3 
    WHERE song = 'All Hands Against His Own'
```

In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "SELECT song, firstName, lastName \
          FROM music_user_songAHAHO \
          WHERE song = 'All Hands Against His Own' "
    

#### Next, we create the table music_user_songAHAHO, and insert data into the table. 
#### Then, we execute the query to validate the results.

In [17]:
# Create table3
session.execute("DROP TABLE IF EXISTS music_user_songAHAHO ")
table3_creation = """CREATE TABLE IF NOT EXISTS music_user_songAHAHO """
table3_creation += "(song text, userId int, firstName text, lastName text, \
                    PRIMARY KEY (song, userId))"

session.execute(table3_creation)

In [18]:
# Insert table3
query_insert3 = "INSERT INTO music_user_songAHAHO (song, userId, firstName, lastName) "
query_insert3 += " VALUES(%s, %s, %s, %s)"

file = 'event_datafile_new.csv' 
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute(query_insert3, (line[9], int(line[10]), line[1], line[4]))

In [19]:
rows = session.execute(query3)

# print("{:<14s} {:<14s} {:<14s}".format('First Name', 'Last Name', 'Song'))
# for row in rows:
#     print("{:<14s} {:<14s} {:<14s}".format(row.firstname, row.lastname, row.song))

tbl = PrettyTable(['First Name', 'Last Name', 'Song'])
for row in rows:
    tbl.add_row([row.firstname, row.lastname, row.song])
print(tbl)

+------------+-----------+---------------------------+
| First Name | Last Name |            Song           |
+------------+-----------+---------------------------+
| Jacqueline |   Lynch   | All Hands Against His Own |
|   Tegan    |   Levine  | All Hands Against His Own |
|    Sara    |  Johnson  | All Hands Against His Own |
+------------+-----------+---------------------------+


### Drop the tables before closing out the sessions

In [20]:
# Drop the table before closing out the sessions

In [21]:
session.execute("DROP TABLE IF EXISTS music_app_history")
session.execute("DROP TABLE IF EXISTS music_user_session")
session.execute("DROP TABLE IF EXISTS music_user_songAHAHO")

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()